In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!ls '/content/gdrive/My Drive/images for 490/cars_train'

1    11   120  131  142  153  164  175	186  2	 30  41  52  63  74  85  96
10   110  121  132  143  154  165  176	187  20  31  42  53  64  75  86  97
100  111  122  133  144  155  166  177	188  21  32  43  54  65  76  87  98
101  112  123  134  145  156  167  178	189  22  33  44  55  66  77  88  99
102  113  124  135  146  157  168  179	19   23  34  45  56  67  78  89
103  114  125  136  147  158  169  18	190  24  35  46  57  68  79  9
104  115  126  137  148  159  17   180	191  25  36  47  58  69  8   90
105  116  127  138  149  16   170  181	192  26  37  48  59  7	 80  91
106  117  128  139  15	 160  171  182	193  27  38  49  6   70  81  92
107  118  129  14   150  161  172  183	194  28  39  5	 60  71  82  93
108  119  13   140  151  162  173  184	195  29  4   50  61  72  83  94
109  12   130  141  152  163  174  185	196  3	 40  51  62  73  84  95


# DenseNet Model

## Imports

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K

Using TensorFlow backend.


In [6]:
K.set_learning_phase(1)

In [0]:
img_width, img_height = 224, 224
# Number of training samples
nb_train_samples = 6074
# Number of Validation samples
nb_validation_samples = 1983
epochs = 10
batch_size = 32
# Number of classes
n_classes = 196

In [11]:
# Assign training and validation directory variables
train_data_dir = '/content/gdrive/My Drive/images for 490/cars_train'
validation_data_dir = '/content/gdrive/My Drive/images for 490/cars_test'

# Image augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 6074 images belonging to 196 classes.
Found 1983 images belonging to 196 classes.


In [0]:
# '../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5'
# Alternative to the below to try: DenseNet121(include_top=True, weights='imagenet', input_tensor=None, classes=1000)
def build_model():
    base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights='imagenet',
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
      layer.trainable = True

    x = base_model.output
    x = Dense(1000, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    x = Dense(500, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [16]:
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse'])












29089792/29084464 [==============================] - 1s 0us/step




In [0]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [19]:
model_history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
190/190 [==============================] - 2136s 11s/step - loss: 151.6021 - acc: 0.0056 - mean_squared_error: 0.0051 - val_loss: 36.1916 - val_acc: 0.0091 - val_mean_squared_error: 0.0051
Epoch 2/10
190/190 [==============================] - 267s 1s/step - loss: 16.9912 - acc: 0.0082 - mean_squared_error: 0.0051 - val_loss: 8.4475 - val_acc: 0.0090 - val_mean_squared_error: 0.0051
Epoch 3/10
190/190 [==============================] - 265s 1s/step - loss: 7.2148 - acc: 0.0083 - mean_squared_error: 0.0051 - val_loss: 6.7535 - val_acc: 0.0090 - val_mean_squared_error: 0.0051
Epoch 4/10
190/190 [==============================] - 265s 1s/step - loss: 6.7228 - acc: 0.0082 - mean_squared_error: 0.0051 - val_loss: 6.7342 - val_acc: 0.0096 - val_mean_squared_error: 0.0051
Epoch 5/10
190/190 [==============================] - 266s 1s/step - loss: 6.7462 - acc: 0.0082 - mean_squared_error: 

# NASNet